# Federated FedProx PyTorch MNIST Tutorial
The only difference between this notebook and Federated_Pytorch_MNIST_Tutorial.ipynb is overriding of the `train_epoch` function in model definition. [See details](#FedProx)


In [ ]:
#Install dependencies if not already installed
!pip install torch torchvision

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet
import random
import warnings
warnings.filterwarnings('ignore')
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(10)


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [ ]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [ ]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

# FedProx

In [ ]:
from openfl.utilities.optimizers.torch import FedProxOptimizer

In [ ]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
    def train_epoch(self, batch_generator):
        from openfl.federated.task import PyTorchTaskRunner
        self.optimizer.set_old_weights([p for p in self.parameters()])
        return PyTorchTaskRunner.train_epoch(self, batch_generator)

    
optimizer = lambda x: FedProxOptimizer(x, lr=1e-3, mu=0.1)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.binary_cross_entropy_with_logits(input=output,target=target.float())

In [ ]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [ ]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [ ]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

We can see the current plan values by running the `fx.get_plan()` function

In [ ]:
 #Get the current values of the plan. Each of these can be overridden
import json
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [ ]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(
    collaborators,
    {
        'aggregator.settings.rounds_to_train': 5,
        'collaborator.settings.opt_treatment': 'CONTINUE_GLOBAL',
    }
)

In [ ]:
#Save final model
final_fl_model.save_native('final_pytorch_model')

# FedProxAdam

In [ ]:
classes = 10
collaborator_num = 300
NUM_USER = collaborator_num

def one_hot(labels, classes):
    return np.eye(classes)[labels]


def softmax(x):
    ex = np.exp(x)
    sum_ex = np.sum(np.exp(x))
    return ex/sum_ex


def generate_synthetic(alpha, beta, iid):

    dimension = 60
    NUM_CLASS = 10

    samples_per_user = np.random.lognormal(4, 2, (NUM_USER)).astype(int) + 50
    num_samples = np.sum(samples_per_user)

    X_split = [[] for _ in range(NUM_USER)]
    y_split = [[] for _ in range(NUM_USER)]

    #### define some eprior ####
    mean_W = np.random.normal(0, alpha, NUM_USER)
    mean_b = mean_W
    B = np.random.normal(0, beta, NUM_USER)
    mean_x = np.zeros((NUM_USER, dimension))

    diagonal = np.zeros(dimension)
    for j in range(dimension):
        diagonal[j] = np.power((j+1), -1.2)
    cov_x = np.diag(diagonal)

    for i in range(NUM_USER):
        if iid == 1:
            mean_x[i] = np.ones(dimension) * B[i]  # all zeros
        else:
            mean_x[i] = np.random.normal(B[i], 1, dimension)

    if iid == 1:
        W_global = np.random.normal(0, 1, (dimension, NUM_CLASS))
        b_global = np.random.normal(0, 1,  NUM_CLASS)

    for i in range(NUM_USER):

        W = np.random.normal(mean_W[i], 1, (dimension, NUM_CLASS))
        b = np.random.normal(mean_b[i], 1,  NUM_CLASS)

        if iid == 1:
            W = W_global
            b = b_global

        xx = np.random.multivariate_normal(
            mean_x[i], cov_x, samples_per_user[i])
        yy = np.zeros(samples_per_user[i])

        for j in range(samples_per_user[i]):
            tmp = np.dot(xx[j], W) + b
            yy[j] = np.argmax(softmax(tmp))

        X_split[i] = xx.tolist()
        y_split[i] = yy.tolist()

#         print("{}-th users has {} exampls".format(i, len(y_split[i])))

    return X_split, y_split


class SyntheticFederatedDataset(FederatedDataSet):
    def __init__(self, batch_size=1, num_classes=None, **kwargs):
        X, y = generate_synthetic(0.0, 0.0, 0)
        X = [np.array([np.array(sample).astype(np.float32)
                      for sample in col]) for col in X]
        y = [np.array([np.array(one_hot(int(sample), classes))
                      for sample in col]) for col in y]
        self.X_train_all = np.array([col[:int(0.9 * len(col))] for col in X])
        self.X_valid_all = np.array([col[int(0.9 * len(col)):] for col in X])
        self.y_train_all = np.array([col[:int(0.9 * len(col))] for col in y])
        self.y_valid_all = np.array([col[int(0.9 * len(col)):] for col in y])
        super().__init__(self.X_train_all[0], self.y_train_all[0], self.X_valid_all[0],
                         self.y_valid_all[0], batch_size, num_classes)

    def split(self, num_collaborators, shuffle=True, equally=False):
        return [
            FederatedDataSet(
                self.X_train_all[i],
                self.y_train_all[i],
                self.X_valid_all[i],
                self.y_valid_all[i],
                batch_size=self.batch_size,
                num_classes=self.num_classes
            ) for i in range(num_collaborators)
        ]

In [ ]:
from openfl.utilities.optimizers.torch import FedProxAdam 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(60, 100)
        self.linear2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        return x

    def train_epoch(self, batch_generator):
        from openfl.federated.task import PyTorchTaskRunner
        self.optimizer.set_old_weights(
            [p.clone().detach() for p in self.parameters()])
        return PyTorchTaskRunner.train_epoch(self, batch_generator)


def optimizer(x): return FedProxAdam(x, lr=1e-3, mu=0.01)
# optimizer = lambda x: torch.optim.Adam(x, lr=1e-3)


def cross_entropy(output, target):
    """Binary cross-entropy metric
     """
    return F.cross_entropy(output, torch.max(target, 1)[1])
#     return F.binary_cross_entropy_with_logits(input=output,target=target.float())


In [ ]:
fl_data = SyntheticFederatedDataset(batch_size=32, num_classes=classes)
#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [ ]:
collaborator_models = fl_model.setup(num_collaborators=collaborator_num,device='cpu')
collaborators = {f'col{i}':collaborator_models[i] for i in range(collaborator_num)}#, 'three':collaborator_models[2]}

In [ ]:
a = np.argmax(collaborators['col3'].data_loader.y_valid, axis =1)
import matplotlib.pyplot as plt
plt.hist(a)
collaborator_models[1].data_loader.y_valid.shape

We can see the current plan values by running the `fx.get_plan()` function

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [ ]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(
    collaborators,
    {
        'aggregator.settings.rounds_to_train': 20,
        'collaborator.settings.opt_treatment': 'CONTINUE_GLOBAL',
    }
)

In [ ]:
#Save final model
final_fl_model.save_native('final_pytorch_model')